# Change Point


In [8]:
import matplotlib.pyplot as plt
import time
import pandas as pd
import seaborn as sns
import numpy as np
import math

import sys
import os

# 1. Lấy đường dẫn của thư mục 'Notebook'
current_dir = os.getcwd() 

# 2. Lấy đường dẫn của thư mục Project_Root (Thư mục cha của 'Notebook' và 'Module')
# Đây là thư mục Project_Root/
project_root_dir = os.path.dirname(current_dir)

# 3. Thêm Project_Root vào sys.path
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)
    
print(f"Đã thêm đường dẫn gốc: {project_root_dir}")

# Lệnh import này sẽ hoạt động sau khi thêm Project_Root vào sys.path
from Module.Load_Data import DataLoader, CleanDataLoader
from Module.Processor_Data import DataProcessor
from Module.Analysis import Analysis
from Module.ANOVA_ttest import ANOVA_ttest
from Module.ChangePoint import ChangePointPreparer
from Module.ChangePointDetector import ChangePointDetector

Đã thêm đường dẫn gốc: c:\Users\ADMIN\Documents\PythonProject


In [ ]:
import pandas as pd
import numpy as np

# Lưu ý: Class này sẽ hoạt động khi được gọi từ Notebook đã setup đường dẫn
# Nên ta vẫn import từ 'Module' bình thường.
try:
    from Module.Load_Data import CleanDataLoader
except ImportError:
    # Fallback cho trường hợp IDE báo lỗi đỏ (không ảnh hưởng khi chạy thật từ Notebook)
    import sys
    import os
    sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))
    from Module.Load_Data import CleanDataLoader

class ChangePointPreparer:
    """
    Chuẩn bị dữ liệu cho bài toán Change Point Detection.
    """
    def __init__(self, clean_loader: CleanDataLoader):
        self.loader = clean_loader

    def _transform_to_timeseries(self, df: pd.DataFrame, series_id: str, metric: str = "mean") -> pd.DataFrame:
        if df is None or df.empty:
            return pd.DataFrame()
        
        # Kiểm tra cột metric
        if metric not in df.columns:
            # Fallback: lấy cột số đầu tiên không phải nam_hoc
            numeric_cols = df.select_dtypes(include=np.number).columns.difference(['nam_hoc'])
            if len(numeric_cols) > 0:
                metric = numeric_cols[0]
            else:
                return pd.DataFrame()
            
        df_out = df[["nam_hoc", metric]].copy()
        df_out.columns = ["year", "value"]
        df_out["series_id"] = series_id
        return df_out[["year", "series_id", "value"]].sort_values("year").reset_index(drop=True)

    def get_subject_series(self, subjects: list[str], metric: str = "mean") -> pd.DataFrame:
        results = []
        for subj in subjects:
            try:
                df = self.loader.get_subject_data(subject=subj, kind="analysis")
                ts = self._transform_to_timeseries(df, series_id=subj, metric=metric)
                results.append(ts)
            except Exception:
                continue
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame(columns=["year", "series_id", "value"])

    def get_block_series(self, blocks: list[str], metric: str = "mean") -> pd.DataFrame:
        results = []
        for blk in blocks:
            try:
                df = self.loader.get_block_data(block=blk, kind="analysis")
                ts = self._transform_to_timeseries(df, series_id=blk, metric=metric)
                results.append(ts)
            except Exception:
                continue
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame(columns=["year", "series_id", "value"])

    def get_province_series(self, provinces: list[str], metric: str = "mean") -> pd.DataFrame:
        results = []
        for prov in provinces:
            try:
                df = self.loader.get_province_data(province=prov, kind="analysis")
                ts = self._transform_to_timeseries(df, series_id=prov, metric=metric)
                results.append(ts)
            except Exception:
                continue
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame(columns=["year", "series_id", "value"])

## 1. Data


<!-- – Từ Subject_Data, Block_Data, Province_Data tạo các chuỗi thời gian ngắn
(2023–2025):
* xmon,year: mean điểm theo môn.
* xblock,year: mean/tổng điểm theo khối.
* xprovince,year: mean tổng điểm theo tỉnh/thành.
– Chuẩn hoá thành các DataFrame dạng:
year, series_id, value
để dễ dàng áp dụng nhiều thuật toán change point trên cùng một cấu trúc. -->



In [ ]:
# – Từ Subject_Data, Block_Data, Province_Data tạo các chuỗi thời gian ngắn
# (2023–2025):
# * xmon,year: mean điểm theo môn.
# * xblock,year: mean/tổng điểm theo khối.
# * xprovince,year: mean tổng điểm theo tỉnh/thành.
# – Chuẩn hoá thành các DataFrame dạng:
# year, series_id, value
# để dễ dàng áp dụng nhiều thuật toán change point trên cùng một cấu trúc


# Ví dụ sử dụng:
change_point_preparer = ChangePointPreparer(clean_loader=CleanDataLoader())
subjects = ["Toan", "Van", "Anh"]
subject_series = change_point_preparer.get_subject_series(subjects, metric="mean")
blocks = ["A00", "A01", "B00"]
block_series = change_point_preparer.get_block_series(blocks, metric="mean")
provinces = ["HaNoi", "ThanhphoHoChiMinh", "ĐaNang"]
province_series = change_point_preparer.get_province_series(provinces, metric="mean")

print(subject_series)
print(block_series)
print(province_series)

KeyError: 1

## 2. Change Point Detection

## 3. Comparison of Methods

## 4. Visualization